## Calculating event duration for heatwaves at a single grid point 

In [1]:
import xarray as xr
import pandas
import glob
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from scipy import stats
from scipy.stats import ttest_ind 

In [2]:
lon = 115.25
lat = -33.0

In [3]:
thw = xr.open_dataset('/g/data/e14/cp3790/Charuni/Tasmania/aus-coastal-sev.nc').sel(latitude=lat, longitude=lon)
mhw = xr.open_dataset('/g/data/e14/cp3790/Charuni/Tasmania/aus-ocean-sev-2.nc').sel(latitude=lat, longitude=lon)

In [4]:
thw_sev = thw.severity
thw_days = xr.where(thw_sev>1, 1, 0)

In [5]:
mhw_sev = mhw.severity
mhw_days = xr.where(mhw_sev>1, 1, 0)

In [6]:
add = thw_days + mhw_days
co = xr.where(add==2, 1, 0)   # when co==1, it represents a co-occurring event 

In [29]:
def event_durations(event_mask, dim='time'):
    """
    Returns the lengths of events marked by 'event_mask'. Where 'event_mask' is 
    true an event is deemed active
    """
    event_stats = []
    
    assert event_mask.ndim == 1, "Only 1d arrays are implemented"

    # Loop over each entry, adding records for events to 'event_stats'
    current_event = None
    for i in range(event_mask.sizes[dim]):
        event_active = event_mask.isel({dim:i})
    
        if event_active:
            if not current_event:
                # A new event
                current_event = {'start': event_mask[dim].data[i], 'duration': 1}
            else:
                # An existing event
                current_event['duration'] += 1
        else:
            if current_event:
                # Event has finished, add to the record array
                event_stats.append(current_event)
                current_event = None

    if current_event:
        # Event active at the end
        event_stats.append(current_event)

    return pandas.DataFrame.from_records(event_stats, index='start')

In [32]:
co_events = event_durations(co, dim='time')

In [33]:
thw_only = event_durations(thw_days.where(co==0, drop=True), dim='time')